# Project 3. Cross Entropy
- Calculate cross entropy of each language model.
- Use `NLRW1900000011.json` as training data, `NLRW1900000020.json` `WARW1900003745.json`as test data,
  - `NLRW1900000020.json` is an newspaper article like training data, and `WARW1900003745.json` is a part of "The Chronicles of Narnia" so the genre of test dataset is far apart from each other.
- Implement Unigram, Bigram, Trigram model from training data and calculate cross entropy using above test data.
- Calcaulate difference (H(P,m) - H(p))
- Tips
  - In training corpus, entropy == cross entropy so the difference is 0.
  - Unigram model calculate entropy as below.
    - H(X) = -p(가) log2 p(가) -p(각) log2 p(각) ...- p(힣) log2 p(힣)
  - p(X) : The probability of each word in test corpus.
  - logp(m) : The probability of the model calculated from test corpus.
  - Using ADD-1 smoothing to cope with n-gram which is not included in training corpus but exists in test corpus.

### Entropy

In Information Theory, entropy (denoted $H(X)$) of a random variable X is the expected log probabiltiy:

\begin{equation}
    H(X) = - \sum P(x)log_2 P(x)
\end{equation}

and is a **measure of uncertainty.**


### Cross Entropy

The cross entropy, H(p,m), of a true distribution **p** and a model distribution **m** is defined as:

\begin{equation}
    H(p,m) = - \sum_{x} p(x) log_2 m(x)
\end{equation}

The lower the cross entropy is the closer it is to the true distribution.

## 1.  Data Loading and Preprocessing

### 1.1 Split paragraph with kss
- If the length of splitted sentence is over 3, include it to sentence list.

In [1]:
import kss

def sentence_splitter(paragraph):
    sentence_list = [f"{s}" for s in kss.split_sentences(paragraph) if len(s) > 3 ]
    return sentence_list

### 1.2. Cleansing sentence
- Cleansing all words except '가~힣'

In [3]:
import re

def cleansing_special(sentence):
    sentence = re.sub("[^가-힣]", " ", sentence)
    sentence = re.sub(' +', ' ', sentence)
    sentence = sentence.strip()
    return sentence

### 1.3. Data Loading
- Loading paragrap from json file's 'document' list and split it to sentence format.

In [5]:
import json

In [6]:
def data_loadeing(filename):
    sentences = []
    doc = json.load(open(filename, 'rt', encoding='UTF8'))['document']
    
    for i in range(len(doc)):
        tmp = doc[i]['paragraph']

        for j in range(len(tmp)):
            sentences.extend(sentence_splitter(doc[i]['paragraph'][j]['form']))

    return sentences

In [7]:
train_filename = '{PATH}/NLRW1900000011.json' # train data set
test1_filename = '{PATH}/NLRW1900000020.json' # test data set #1
test2_filename = '{PATH}/WARW1900003745.json' # test data set #2

data_list = ['train','test1','test2']

sentences = {}

sentences[data_list[0]] = data_loadeing(train_filename)

sentences[data_list[1]] = data_loadeing(test1_filename)

sentences[data_list[2]] = data_loadeing(test2_filename)

In [8]:
sentences[data_list[2]][0:20]

['나니아의 국경에서',
 '여기는 나니아의 국경이다.',
 '반 세기가 넘도록 우리 세계의 수많은 독자들이 울고 웃고 뒹굴며 모험을 즐긴 바로 그 나니아가 여러분의 코앞에 있다.',
 '이곳은 알려진 대로 판타지의 세계이다.',
 '진정한 판타지는 이상적인 세계에 대한 갈망을 일으키고 실제 세계에 새로운 차원의 깊이를 제공하는 것이라고, 작가는 말했다.',
 '나니아는 바로 그런 나라이다.',
 '게으름과 욕심, 슬픔과 절망, 소외감과 공포를 가지고 오면 희망과 용기, 사랑과 웃음, 자신감과 인내심을 값없이 드리겠다.',
 '여권으로 호기심만 갖추고 있다면 이 나라에 들어오는 조건은 별로 까다롭지 않다.',
 '그러나 여러분이 입국하기 전에 챙겨 오면 좋을 것을 지금 알려 드리겠다.',
 '첫째는 상상력이다.',
 '이런 종류의 판타지 나라를 만든 사람들이 다 그렇듯이, 《나니아 연대기》를 지은 C. S. 루이스는 상상력이 보통이 아닌 작가였다.',
 '병약했던 어린 시절, 북아일랜드의 수도인 벨파스트의 큰 집을 구석구석을 돌아다니며 환상의 날개를 펼쳤다.',
 '그 집은 동물의 왕국이 되기도 했고, 인도가 되기도 했다.',
 '책을 좋아했던 그는 세계 여러 나라의 신화와 전설, 선대 동화작가들의 작품 세계에 푹 빠져 살았다.',
 '그렇게 키워낸 상상력으로 이토록 생기 넘치는 나니아를 만들어낸 것이다.',
 '상상력이란, 보이는 것을 보이는 그대로, 주어진 것을 주어진 그대로 받아들이지 않고 자기만의 생각의 힘으로 변화시키고, 없는 것을 만들어내 덧붙여서, 새로운 세계를 창조하는 힘이다.',
 '그런 상상력으로 세워진 판타지 나라는 상상력 풍부한 여행객일수록 잘 이해하고 즐기는 법이니 상상력을 잊지 말고 챙기시길 바란다.',
 '둘째는 유머 감각이다.',
 '루이스는 유머 감각이 뛰어났고, 유머를 무엇보다 소중히 여기는 사람이었다.',
 '천국과 지옥의 차이를 유머가 있고 없고로 설명할 정도였으니.']

### 1.4. Cleansing sentences
- sentences to cleansed_sentence

In [9]:
def cleansing_sentences(sentences):
    cleansed_sentences = []
    for s in sentences:
        cleansed_sentences.append(cleansing_special(s))
    return cleansed_sentences

In [10]:
cleansed_sentences = {}
for data in data_list:
    cleansed_sentences[data] = cleansing_sentences(sentences[data])

### Checking cleansed data

- Double-check for remained special characters

In [11]:
for i in range(1,20):
    print(sentences['test2'][i])
    print(cleansed_sentences['test2'][i])
    print()

여기는 나니아의 국경이다.
여기는 나니아의 국경이다

반 세기가 넘도록 우리 세계의 수많은 독자들이 울고 웃고 뒹굴며 모험을 즐긴 바로 그 나니아가 여러분의 코앞에 있다.
반 세기가 넘도록 우리 세계의 수많은 독자들이 울고 웃고 뒹굴며 모험을 즐긴 바로 그 나니아가 여러분의 코앞에 있다

이곳은 알려진 대로 판타지의 세계이다.
이곳은 알려진 대로 판타지의 세계이다

진정한 판타지는 이상적인 세계에 대한 갈망을 일으키고 실제 세계에 새로운 차원의 깊이를 제공하는 것이라고, 작가는 말했다.
진정한 판타지는 이상적인 세계에 대한 갈망을 일으키고 실제 세계에 새로운 차원의 깊이를 제공하는 것이라고 작가는 말했다

나니아는 바로 그런 나라이다.
나니아는 바로 그런 나라이다

게으름과 욕심, 슬픔과 절망, 소외감과 공포를 가지고 오면 희망과 용기, 사랑과 웃음, 자신감과 인내심을 값없이 드리겠다.
게으름과 욕심 슬픔과 절망 소외감과 공포를 가지고 오면 희망과 용기 사랑과 웃음 자신감과 인내심을 값없이 드리겠다

여권으로 호기심만 갖추고 있다면 이 나라에 들어오는 조건은 별로 까다롭지 않다.
여권으로 호기심만 갖추고 있다면 이 나라에 들어오는 조건은 별로 까다롭지 않다

그러나 여러분이 입국하기 전에 챙겨 오면 좋을 것을 지금 알려 드리겠다.
그러나 여러분이 입국하기 전에 챙겨 오면 좋을 것을 지금 알려 드리겠다

첫째는 상상력이다.
첫째는 상상력이다

이런 종류의 판타지 나라를 만든 사람들이 다 그렇듯이, 《나니아 연대기》를 지은 C. S. 루이스는 상상력이 보통이 아닌 작가였다.
이런 종류의 판타지 나라를 만든 사람들이 다 그렇듯이 나니아 연대기 를 지은 루이스는 상상력이 보통이 아닌 작가였다

병약했던 어린 시절, 북아일랜드의 수도인 벨파스트의 큰 집을 구석구석을 돌아다니며 환상의 날개를 펼쳤다.
병약했던 어린 시절 북아일랜드의 수도인 벨파스트의 큰 집을 구석구석을 돌아다니며 환상의 날개를 펼쳤다

그 집은 동물의 왕국이 되기도 했고, 인도가 되기도 했다.
그 집은 

## 2. Define ngram counter function & Excution
- Using `zip` function to implement n-gram.
- Using `Counter` function from `collections` package to count n-gram.

In [12]:
import time
from collections import Counter

In [13]:
def ngram_counter(sentence_list, n):
    ngram = []
    
    for i, s in enumerate(sentence_list):
        if n == 1:
            ngram.extend(s)
        else:
            ngram.extend(zip(*[s[i:] for i in range(n)]))
            
    return Counter(ngram)

In [14]:
ngram_count={} # Save the number of ngram in dict 

for data in data_list:
    ngram_count[data]={}
    for i in range(4):
        ngram_count[data][i]=ngram_counter(cleansed_sentences[data], i)

### 2.1.2.Smoothing (Add-1) implementation
- Add 1 to every n-gram from training data set.

In [15]:
for n in range(1,4):
    ngram_count['train'][n] = dict(zip(ngram_count['train'][n].keys(),map(lambda x:x[1]+1,
                                                                          ngram_count['train'][n].items())))

- Process n-grams which is not in training dataset.

In [ ]:
for data in data_list[1:]:
    for n in range(1,4):
        for ng in ngram_count[data][n].keys():
            if ng not in ngram_count['train'][n]:
                ngram_count['train'][n][ng]=1  # Initialize to 1 

### 2.1.3. Count all n-grams

In [17]:
ngram_count_total={}
for data in data_list:
    ngram_count_total[data] = {}
    for n in range(1,4):
        ngram_count_total[data][n]=sum(ngram_count[data][n].values())

## 3. Calculate the probability of each n-grams

In [18]:
prob_ngram = {}

for data in data_list:
    prob_ngram[data]={}
    for n in range(1,4):
        ngram=[]
        prob_ngram[data][n] = {}
        for ng in ngram_count[data][n].keys():
            prob_ngram[data][n][ng] = ngram_count[data][n][ng] / ngram_count_total[data][n]

## 4. Calculate Entropy, Cross-entropy, Difference 

In [19]:
import numpy as np
import pandas as pd

In [20]:
raw_data = {'File Name': ['NLRW1900000011.json', '', '', 'NLRW1900000020.json','', '', 'WARW1900003745.json', '', ''],
            'n-gram': ['unigram','bigram','trigram','unigram','bigram','trigram','unigram','bigram','trigram']}

result = pd.DataFrame(raw_data)

result

,File Name,n-gram
0,NLRW1900000011.json,unigram
1,,bigram
2,,trigram
3,NLRW1900000020.json,unigram
4,,bigram
5,,trigram
6,WARW1900003745.json,unigram
7,,bigram
8,,trigram


### 4.1. Entropy 
- Save results to `result['Entropy']`
$$H(p) = -\sum_{i=1}^{n} p(x_i){log} p(x_i)$$

In [21]:
entropy=[]
for data in data_list:
    for n in range(1,4):
        entropy.append(-sum(p*np.log2(p) for p in prob_ngram[data][n].values()))

In [22]:
result['Entropy']=entropy

### 4.2. Cross-entropy 
- Save results to `result['Cross-entropy']`
$$H(p,q) = -\sum_{i=1}^{n} p(x_i)logq(x_i)$$

In [23]:
cross_entropy=[]
for data in data_list:
    for n in range(1,4):
         cross_entropy.append(-sum(prob_ngram[data][n][ng]*np.log2(prob_ngram['train'][n][ng])
                                   for ng in prob_ngram[data][n].keys()))

In [24]:
result['Cross-entropy']=cross_entropy

### 4.3. Difference 

In [25]:
result['Difference'] = result['Cross-entropy'] - result['Entropy']

result

,File Name,n-gram,Entropy,Cross-entropy,Difference
0,NLRW1900000011.json,unigram,6.875718,6.875718,0.000000
1,,bigram,11.854136,11.854136,0.000000
2,,trigram,15.790079,15.790079,0.000000
3,NLRW1900000020.json,unigram,6.912301,6.949638,0.037337
4,,bigram,11.672032,11.956467,0.284435
5,,trigram,15.071285,15.855163,0.783878
6,WARW1900003745.json,unigram,6.777204,7.268261,0.491056
7,,bigram,11.117389,12.806630,1.689241
8,,trigram,14.275727,17.162749,2.887022
